In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
 pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import matplotlib.pyplot as plt
import json


import re
import psycopg2
from sqlalchemy import create_engine


from config import db_password,database,user,host,port


In [5]:



import psycopg2
engine = psycopg2.connect(
    database=database,
    user=user,
    password=db_password,
    host="housing-database.cmjqg2he80uz.us-east-2.rds.amazonaws.com",
    port=port
)






 
 



In [6]:
df = pd.read_sql_query("SELECT * FROM housing ",con=engine)

In [7]:
df.head()

,id,url,region,region_url,price,type,sqfeet,beds,baths,cats_allowed,...,wheelchair_access,electric_vehicle_charge,comes_furnished,laundry_options,parking_options,image_url,description,latitude,longitude,state
0,7047381918,https://cedarrapids.craigslist.org/apa/d/cedar...,cedar rapids,https://cedarrapids.craigslist.org,694.0,apartment,596,1,1.0,1,...,0,0,0,laundry on site,off-street parking,https://images.craigslist.org/01313_dgwpzX0yEI...,www.AlexandraApts.com At Alexandra Apartments...,41.9669,-91.7285,ia
1,7031680107,https://cedarrapids.craigslist.org/apa/d/cedar...,cedar rapids,https://cedarrapids.craigslist.org,615.0,apartment,750,2,1.0,1,...,0,0,0,laundry in bldg,detached garage,https://images.craigslist.org/00202_j7mC8PCyoG...,Very spacious 2 bedroom unit with remodeled ki...,42.0146,-91.6308,ia
2,7031678322,https://cedarrapids.craigslist.org/apa/d/cedar...,cedar rapids,https://cedarrapids.craigslist.org,615.0,apartment,750,2,1.0,1,...,0,0,0,laundry in bldg,detached garage,https://images.craigslist.org/00R0R_8zreipTfGf...,Here at Continental Terrace Apartments we welc...,42.0147,-91.6307,ia
3,7047376663,https://cedarrapids.craigslist.org/apa/d/cedar...,cedar rapids,https://cedarrapids.craigslist.org,759.0,apartment,833,2,1.0,1,...,0,0,0,laundry on site,off-street parking,https://images.craigslist.org/00b0b_lToTMrF94A...,www.AlexandraApts.com At Alexandra Apartments...,41.9669,-91.7285,ia
4,7047381784,https://cedarrapids.craigslist.org/apa/d/cedar...,cedar rapids,https://cedarrapids.craigslist.org,1110.0,apartment,960,2,1.5,1,...,0,0,1,w/d hookups,None,https://images.craigslist.org/00F0F_5kBcK1AXGA...,To schedule a tour We now book our tour appoin...,42.0427,-91.6429,ia


In [ ]:
# Load the data

df = pd.read_csv('housing.csv')
df

In [ ]:
print(df.columns)

In [ ]:
# Drop the null rows
df = df.dropna()

df

In [ ]:
df=df.drop(columns=['id','url','region_url','image_url','description','lat','long'])
df.head()

In [ ]:
region_counts=df.region.value_counts()
region_counts

In [ ]:
region_counts.plot.density(xlim=[0,3000])

The density plot falls off around 500. Based on the plot we can see that there is still a sizable amount of data with the frequency greater than 500. So we will filter out regions that have a frequency less than 500 occurances to ensure all regions we work with have a good amount of data.

In [ ]:
regions_with_enough_data=list(region_counts[region_counts>=500].index)

Next step is filtering the dataframe with regions greater than 500

In [ ]:
region_enough_data_df=df[df.region.isin(regions_with_enough_data) ]
region_enough_data_df

In [ ]:
df=region_enough_data_df

In [ ]:
df['price'].describe()

In [ ]:
range=list(df.price.quantile([0.01,0.99]))
range

In the previous step we have found that values between 1 percentile and 99 percentile. Rest of the values are considered outliers and will be ignored.

In [ ]:
df=df[(df.price >= range[0]) & (df.price <= range[1])]
df

In [ ]:
df.type.value_counts()

Loft, flat, cottage/cabin, in-law, land and assisted living have a very small amount
of data (<1% of the dataset) so we will ignore it.

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
df=pd.get_dummies(data=df, columns=['region','type','laundry_options','parking_options','state'])
df

In [ ]:
# Create our features
from sklearn.model_selection import train_test_split
y = df['price']
#X_with_dummies = pd.get_dummies(df)
#X_with_dummies

X=df.drop(columns="price")
#X=pd.get_dummies(X)

# Create our target


In [ ]:
X

In [ ]:
X.columns.to_list()

In [ ]:
from sklearn.model_selection import train_test_split
# YOUR CODE HERE
X_train, X_test, y_train, y_test=train_test_split(X,y, random_state=1)
#Figure out how to fix stratify

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_test)
print(y_pred.shape)

In [ ]:
from sklearn import metrics

# RMSE
print(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
